# Sťahovanie dát flares z datasetu flares3.csv

Sťahovanie flares dát z datasetu flares3.csv 5 pred + PEAK+ 5 po


* sťahovanie dát typu flares do priečinku data2

* triedenie obrázkov flares do priečinkov train, test a valid (priečinok flares)

In [7]:
pip install h5py==3.7.0 astropy==6.1.4 parfive==2.1.0 Glymur==0.13.6 sunpy==6.0.3 numpy==1.24.4 hvpy==1.1.0 aiapy==0.7.4 opencv-python==4.10.0.84 ultralytics torch

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.2.0
aioftp                        0.23.1
aiohttp                       3.8.1
aiosignal                     1.2.0
alembic                       1.8.1
altair                        4.2.0
annotated-types               0.7.0
anyio                         3.6.1
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
astropy                       5.3.4
astropy-iers-data             0.2024.11.11.0.32.38
asttokens                     2.0.8
astunparse                    1.6.3
async-generator               1.10
async-timeout                 4.0.2
attrs                         22.1.0
Babel                         2.10.3
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
beautifulsoup4                4.11.1
bleach                        5.0.1
blinker                       1.4
bokeh                         2.4.3
Bottleneck                   

sťahovanie dát zo súboru flares.csv -> nesprávne hodnoty latitude a longitude

In [ ]:
# kniznice
import pandas as pd
import os
from sunpy.map import Map
import hvpy
import cv2
from datetime import datetime, timedelta
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits

In [ ]:
dataFlares = pd.read_csv("csv's/flares.csv")

In [41]:
dataFlares.head()

,class,level,time,latitude,longitude
0,C,5.7,01 May 2010 at 01:39,173.078231,23.731655
1,C,3.6,04 May 2010 at 16:29,227.097784,40.917763
2,C,2.3,05 May 2010 at 07:16,226.567829,40.925556
3,C,8.8,05 May 2010 at 11:52,226.231976,40.922768
4,M,1.2,05 May 2010 at 17:19,225.460251,40.894318


In [42]:
base_folder = "./flares"
segmented_folder = "./flares/segmented"
class_folders = {'C': 'C', 'M': 'M', 'X': 'X'}
for cls in class_folders.values():
    os.makedirs(os.path.join(base_folder, cls), exist_ok=True)
    os.makedirs(os.path.join(segmented_folder, cls), exist_ok=True)

In [52]:
# test - problem s latitude

# ziskaj source ID pre SDO AIA 1600
data_sources = hvpy.getDataSources()
source_id = data_sources['SDO']['AIA']['1600']['sourceId']

# spracuj kazdy riadok v subore flares.csv
for index, row in dataFlares.iterrows():
    event_class = row['class']
    if event_class not in class_folders:
        continue  # preskoc ak trieda nie je 'C', 'M' alebo 'X'
    
    level = row['level']
    tpeak = datetime.strptime(row['time'].strip(), '%d %b %Y at %H:%M')

    # prekonvertuj latitude a longitude na float
    img_lat = float(row['latitude'])
    img_lon = float(row['longitude'])

    # zaokruhli latitude na cele cislo
    img_lat = round(img_lat)

    # uprav longitude do rozsahu 0 - 360
    img_lon = img_lon % 360

    # definuj suradnice pre vyrez pomocou heliografickych hranic
    top_right = SkyCoord(lon=(img_lon - 20) * u.deg, lat=(img_lat + 20) * u.deg, frame='heliographic_stonyhurst')
    bottom_left = SkyCoord(lon=(img_lon + 20) * u.deg, lat=(img_lat - 20) * u.deg, frame='heliographic_stonyhurst')

    # cesty k priecinkom
    class_folder = os.path.join(base_folder, class_folders[event_class])
    cutout_class_folder = os.path.join(segmented_folder, class_folders[event_class])

    # vytvor priecinky ak neexistuju
    os.makedirs(class_folder, exist_ok=True)
    os.makedirs(cutout_class_folder, exist_ok=True)

    # definuj nazov suboru
    filename_suffix = f"{tpeak.strftime('%Y%m%d_%H%M')}_{event_class}{level}"
    filename = f"{filename_suffix}_PEAK.jp2"
    jp2_path = os.path.join(class_folder, filename)
    
    # stiahni a uloz obrazok
    if not os.path.exists(jp2_path):
        try:
            image_content = hvpy.getJP2Image(date=tpeak.strftime('%Y-%m-%dT%H:%M'), sourceId=source_id)
            if image_content:
                with open(jp2_path, 'wb') as f:
                    f.write(image_content)
            else:
                print(f"No content retrieved for {tpeak}. Skipping.")
                continue
        except Exception as e:
            print(f"Error retrieving image for {tpeak}: {e}")
            continue

    # spracuj vyrez z obrazka
    if os.path.exists(jp2_path):
        try:
            file = Map(jp2_path)
            cutout_filename = f"{filename_suffix}_cutout.fits"
            cutout_path = os.path.join(cutout_class_folder, cutout_filename)

            # vytvor vyrez a uloz ho
            aia_sub = file.submap(bottom_left, top_right=top_right)
            aia_sub.save(cutout_path, overwrite=True)
            os.remove(jp2_path)  # volitelne odstran originalny JP2 subor
        except Exception as e:
            print(f"Error saving cutout {cutout_path}: {e}")
            continue

ValueError: Latitude angle(s) must be within -90 deg <= angle <= 90 deg, got 103.0 deg

# flares3.csv - stahovanie 5 pred a 5 po do C, M a X (data2)

In [2]:
pip install h5py==3.7.0 astropy==6.1.4 parfive==2.1.0 Glymur==0.13.6 sunpy==6.0.3 numpy==1.24.4 hvpy==1.1.0 hvpy==1.1.0 aiapy==0.7.4 opencv-python==4.10.0.84 ultralytics torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 9.0 MB/s eta 0:00:00:00:01:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 18.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 12.1 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 39.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 501.9/501.9 kB 24.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 11.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.8/914.8 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.7/738.7 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.9 MB/s eta 0:00:00:00:01
  Usi

In [ ]:
import os
import pandas as pd
from sunpy.map import Map
from astropy.coordinates import SkyCoord
import astropy.units as u
import hvpy
from aiapy.calibrate import normalize_exposure
from datetime import datetime, timedelta
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

def download_and_process_images(flares_file, output_folder):
    # nacitanie dat o erupciach
    flares_data = pd.read_csv(flares_file, header=0)
    
    def filter_valid_dates(date_str):
        try:
            date = datetime.strptime(date_str, '%d %B %Y at %H:%M')
            return date
        except ValueError:
            return False

    # ponechanie len riadkov s platnym datumom
    flares_data = flares_data[flares_data['time'].apply(filter_valid_dates)]

    # vytvorenie priecinkov pre jednotlive triedy
    class_folders = {"C": os.path.join(output_folder, "C"),
                     "M": os.path.join(output_folder, "M"),
                     "X": os.path.join(output_folder, "X")}
    for folder in class_folders.values():
        os.makedirs(folder, exist_ok=True)

    # ziskanie source ID pre SDO AIA 1600
    data_sources = hvpy.getDataSources()
    source_id = data_sources['SDO']['AIA']['1600']['sourceId']

    for index, row in flares_data.iterrows():
        flare_class = row['class'][0]  # ziskaj triedu (C, M, X)
        if flare_class not in class_folders:
            continue

        try:
            flare_time = datetime.strptime(row['time'], '%d %B %Y at %H:%M')
        except ValueError as e:
            print(f"Skipping row {index} due to time format error: {e}")
            continue

        flare_lat = row['latitude']
        flare_lon = row['longitude']

        # iterovanie cez casy: 5 minut pred a po PEAK, bez PEAK samotneho
        for minute_offset in range(-5, 6):
            if minute_offset == 0:
                continue  # preskoc PEAK cas

            current_time = flare_time + timedelta(minutes=minute_offset)
            date_str = current_time.strftime('%Y-%m-%dT%H:%M:%S')

            try:
                jp2_data = hvpy.getJP2Image(date=date_str, sourceId=source_id)
            except Exception as e:
                print(f"Error retrieving JP2 image for {date_str}: {e}")
                continue

            # docasny JP2 subor
            filename_jp2 = f"temp_{current_time.strftime('%Y%m%d_%H%M')}.jp2"
            with open(filename_jp2, 'wb') as f:
                f.write(jp2_data)

            try:
                full_map = Map(filename_jp2)
                full_map = normalize_exposure(full_map)
            except Exception as e:
                print(f"Error creating or normalizing Map from {filename_jp2}: {e}")
                os.remove(filename_jp2)
                continue
           # transformácia do helioprojective-cartesian (HPC)
            try:
                point_in_stonyhurst = SkyCoord(flare_lon * u.deg, flare_lat * u.deg, frame='heliographic_stonyhurst', obstime=current_time)
                point_in_hpc = point_in_stonyhurst.transform_to(full_map.coordinate_frame)

                dx = 500 * u.arcsec   # horizontálny rozsah
                dy = 520 * u.arcsec # vertikálny rozsah
                bottom_left = SkyCoord(point_in_hpc.Tx - dx, point_in_hpc.Ty - dy, frame=full_map.coordinate_frame)
                top_right = SkyCoord(point_in_hpc.Tx + dx, point_in_hpc.Ty + dy, frame=full_map.coordinate_frame)
                cutout_map = full_map.submap(bottom_left=bottom_left, top_right=top_right)
            except Exception as e:
                print(f"Error creating submap for {date_str}: {e}")
                os.remove(filename_jp2)
                continue

            # ulozenie suboru vo formate FITS
            filename_fits = f"{current_time.strftime('%Y%m%d_%H%M')}_{row['class']}.fits"
            output_path = os.path.join(class_folders[flare_class], filename_fits)

            if os.path.exists(output_path):
                print(f"File {output_path} already exists. Skipping.")
                os.remove(filename_jp2)
                continue

            try:
                cutout_map.save(output_path, overwrite=True)
                print(f"Saved cutout to {output_path}")
            except Exception as e:
                print(f"Error saving FITS file {output_path}: {e}")

            os.remove(filename_jp2)

    print("Images downloaded")

if __name__ == "__main__":
    flares_file = "csv's/flares3.csv"
    output_folder = "data2"
    download_and_process_images(flares_file, output_folder)

File data2/C/20131117_0102_C.fits already exists. Skipping.
File data2/C/20131117_0103_C.fits already exists. Skipping.
File data2/C/20131117_0104_C.fits already exists. Skipping.
File data2/C/20131117_0105_C.fits already exists. Skipping.
File data2/C/20131117_0106_C.fits already exists. Skipping.
File data2/C/20131117_0108_C.fits already exists. Skipping.
File data2/C/20131117_0109_C.fits already exists. Skipping.
File data2/C/20131117_0110_C.fits already exists. Skipping.
File data2/C/20131117_0111_C.fits already exists. Skipping.
File data2/C/20131117_0112_C.fits already exists. Skipping.
File data2/C/20131117_0212_C.fits already exists. Skipping.
File data2/C/20131117_0213_C.fits already exists. Skipping.
File data2/C/20131117_0214_C.fits already exists. Skipping.
File data2/C/20131117_0215_C.fits already exists. Skipping.
File data2/C/20131117_0216_C.fits already exists. Skipping.
File data2/C/20131117_0218_C.fits already exists. Skipping.
File data2/C/20131117_0219_C.fits alread

# konverzia na png

In [2]:
pip install h5py==3.7.0 astropy==6.1.4 parfive==2.1.0 Glymur==0.13.6 sunpy==6.0.3 numpy==1.24.4 hvpy==1.1.0 hvpy==1.1.0 aiapy==0.7.4 opencv-python==4.10.0.84 ultralytics torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 4.1 MB/s eta 0:00:0000:010:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 41.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 54.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 37.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 501.9/501.9 kB 26.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 10.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.6/914.6 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.7/738.7 kB 34.5 MB/s eta 0:00:00
  Using cached p

In [3]:
import os
import shutil
from datetime import datetime, timedelta
from astropy.io import fits
import matplotlib.pyplot as plt

# cesta
source_folder = "data2"

# podpriecinkz
categories = ["C", "M", "X"]

# konverzia FITS na PNG, nasledne odstranenia FITS
for category in categories:
    category_path = os.path.join(source_folder, category)
    if not os.path.exists(category_path):
        continue

    for file in os.listdir(category_path):
        if file.endswith(".fits"):
            fits_path = os.path.join(category_path, file)
            png_path = fits_path.replace(".fits", ".png")

            # konverzia FITS na PNG
            try:
                with fits.open(fits_path, ignore_missing_simple=True, verify='silentfix') as hdul:
                    if hdul[0].data is None:
                        print(f"Skipping {fits_path} - No image data found.")
                        continue
                    data = hdul[0].data
                    plt.figure(figsize=(5, 5))
                    plt.imshow(data, cmap="gray", origin="lower")
                    plt.axis("off")
                    plt.savefig(png_path, bbox_inches="tight", pad_inches=0)
                    plt.close()

                # odstranenie FITS
                os.remove(fits_path)
            except Exception as e:
                print(f"Error converting {fits_path} to PNG: {e}")

print("Conversion and cleaning of FITS files completed!")

Error converting data2/C/20130103_2141_C.fits to PNG: Header missing END card.
Error converting data2/C/20130103_2142_C.fits to PNG: Header missing END card.


Error converting data2/C/20131118_0138_C.fits to PNG: Empty or corrupt FITS file
Error converting data2/C/20131118_0140_C.fits to PNG: Empty or corrupt FITS file
Error converting data2/C/20131118_0141_C.fits to PNG: Empty or corrupt FITS file
Error converting data2/C/20131118_0142_C.fits to PNG: Empty or corrupt FITS file


Conversion and cleaning of FITS files completed!


# presun do train test valid (priečinok flares)

In [6]:
import os
import shutil
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split

# cesty k priecinkom
source_folder = "data2"
destination_folder = "flares"

# nazvy podpriecinkov
categories = ["C", "M", "X"]
splits = ["train", "test", "valid"]

# vytvor adresarovu strukturu pre experimentFlares2
os.makedirs(destination_folder, exist_ok=True)
for split in splits:
    for category in categories:
        os.makedirs(os.path.join(destination_folder, split, category), exist_ok=True)

# funkcia na zgrupovanie suborov podla udalosti
def group_files_by_events(files):
    events = {}
    for file in files:
        if "PEAK" in file:
            parts = file.split("_")
            peak_time = datetime.strptime(parts[0] + parts[1], "%Y%m%d%H%M")
            event_id = "_".join(parts[:3])  # pouzitie datumu, casu a triedy ako ID udalosti

            # najdenie suborov patriacich k danej udalosti (5 minut pred a po PEAK)
            event_files = []
            for minute_offset in range(-5, 6):
                event_time = peak_time + timedelta(minutes=minute_offset)
                time_str = event_time.strftime("%Y%m%d_%H%M")
                matching_file = next((f for f in files if time_str in f and parts[2] in f), None)
                if matching_file:
                    event_files.append(matching_file)

            events[event_id] = event_files

    return list(events.values())

# kopirovanie suborov a rozdelenie do train, test a valid
for category in categories:
    category_path = os.path.join(source_folder, category)
    if not os.path.exists(category_path):
        continue

    # nacitanie vsetkych suborov v danej kategorii
    files = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]

    # zgrupovanie suborov podla udalosti
    grouped_events = group_files_by_events(files)

    # rozdelenie udalosti na train, test a valid (80 % train, 20 % test, z train 10 % na valid)
    train_events, test_events = train_test_split(grouped_events, test_size=0.2, random_state=42)
    train_events, valid_events = train_test_split(train_events, test_size=0.1, random_state=42)

    # kopirovanie suborov do prislusnych adresarov
    for event_files in train_events:
        for file in event_files:
            shutil.copy(os.path.join(category_path, file), os.path.join(destination_folder, "train", category, file))

    for event_files in test_events:
        for file in event_files:
            shutil.copy(os.path.join(category_path, file), os.path.join(destination_folder, "test", category, file))

    for event_files in valid_events:
        for file in event_files:
            shutil.copy(os.path.join(category_path, file), os.path.join(destination_folder, "valid", category, file))

print("File organization and copying completed!")

File organization and copying completed!


# Graf pre počet obrázkov v pričinkoch train, test a valid jednotlivých tried z datasetu flares.csv (flares)

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

# priecinky
base_path = "./flares"
splits = ["train", "test", "valid"]
classes = ["C", "M", "X"]
colors = {'C': '#157145', 'M': '#8ACE00', 'X': '#63D471'}

# ziskavanie poctov obr
counts = []
for split in splits:
    for cls in classes:
        folder = os.path.join(base_path, split, cls)
        count = len([f for f in os.listdir(folder) if f.endswith(".png")])
        counts.append((split, cls, count))

# priprava dat
labels = []
values = []
bar_colors = []
group_positions = {'train': 0, 'test': 4, 'valid': 8}
bar_positions = []

for i, (split, cls, count) in enumerate(counts):
    base_pos = group_positions[split]
    offset = classes.index(cls)
    bar_positions.append(base_pos + offset)
    values.append(count)
    labels.append(split)
    bar_colors.append(colors[cls])

# vykreslenie grafu
plt.figure(figsize=(10, 6))
bars = plt.bar(bar_positions, values, color=bar_colors)

# pocty nad stlpcami
for bar, value in zip(bars, values):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2.0, height * 1.05, f'{value}', ha='center', va='bottom', fontsize=8)

# popis pre train test a valid
plt.xticks([1, 5, 9], ["Trénovacia množina", "Testovacia množina", "Validačná množina"])

# legenda
legend_labels = [plt.Rectangle((0,0),1,1, color=colors[cls]) for cls in classes]
plt.legend(legend_labels, classes, title='Trieda')

plt.yscale("log")
plt.ylabel("Počet obrázkov")
#plt.title("Počet obrázkov pre jednotlivé triedy (C, M a X) rozdelených na trénovaciu, testovaciu a validačnú množinu")
plt.tight_layout()

# ulozenie grafu
output_path = "outputsData/pocetTrainTestValidFlares.png"  # cesta a nazov suboru
plt.savefig(output_path, bbox_inches='tight')  # ulozi obrazok

#plt.show()